In [ ]:
#import packages
import pandas as pd
import datetime as dt
import numpy as np
import unidecode 
from unidecode import unidecode

import os

In [ ]:
# dtype_mapping = {'Date de création': 'datetime64[D]'}

# raw_list = pd.read_excel('C:/Users/BGE/Google Drive/GIT/listeCreations 2021 to 2024 02 29.xlsx', dtype=dtype_mapping)
raw_list = pd.read_excel('C:/Users/BGE/Google Drive/GIT/listeCreations 2021 to 2024 02 29.xlsx')
raw_list.info()
raw_list.head(50)
raw_list['Date de création'].dtype
# raw_list['Date de création'].astype('<M8[D]')
# raw_list['Date de création'] = pd.to_datetime(raw_list['Date de création'])
# raw_list['Date de création']= raw_list['Date de création'].dt.floor('D')
raw_list['Date de création'] = pd.to_datetime(raw_list['Date de création'], unit='D', origin='1899-12-30')


In [ ]:
raw_list

In [ ]:
#replace "nan" by "unknown"
replace_nan = lambda x: x.replace(np.nan, 'unknown')

columns_to_check = [ column_str for column_str in raw_list.columns if raw_list.columns.dtype == 'object' ]
mask = raw_list.columns.isin(columns_to_check)

# Filter DataFrame colums using the mask
columns_to_check = raw_list.columns[mask]

# Apply the replace_nan function only to columns that are not of type 'object'
raw_list[columns_to_check] = raw_list[columns_to_check].apply(replace_nan)

raw_list


In [ ]:
#concatenate "Nom" and "Prenom"
raw_list['Entrepreneur'] = raw_list['Prénom'] + ' ' + raw_list['Nom']
raw_list.drop(columns=['Prénom', 'Nom'], inplace=True)

#reordered columns
raw_list_col_reordered = ['Lieu accueil', 'Entrepreneur', 'Lieu de suivi', 'Raison sociale', 'Date de création',
       'Type de création', 'Statut fiscal', 'Banque', 'Expert comptable',
       'Phase STARTER Post Création acceptée', 'total anté (h)',
       'total anté Starter (h)', 'ref. Entreprise', 'total h suivi', 'CAPE',
       'Durée (h) consulting', 'Durée (h) formation créa', 'partenaire'
       ]
raw_list = raw_list[raw_list_col_reordered]
raw_list

In [ ]:
# harmonize between tables coaches names

#split ref Entreprise column, join in inverse way firstname / name
raw_list[['Name ref', 'Firstname ref']] = raw_list['ref. Entreprise'].str.rsplit(' ', n=1, expand=True)
raw_list['Conseiller'] = raw_list['Firstname ref'] + ' ' + raw_list['Name ref']

#erase old ref Entreprise column and splitted columns
raw_list.drop(columns=['ref. Entreprise', 'Name ref', 'Firstname ref'])

#reorder columns
raw_list_columns_reordered = ['Lieu accueil', 'Entrepreneur', 'Lieu de suivi', 'Conseiller', 'Raison sociale',
       'Date de création', 'Type de création', 'Statut fiscal', 'Banque',
       'Expert comptable', 'Phase STARTER Post Création acceptée',
       'total anté (h)', 'total anté Starter (h)',
       'total h suivi', 'CAPE', 'Durée (h) consulting',
       'Durée (h) formation créa', 'partenaire']
cleaned_list = raw_list[raw_list_columns_reordered]
cleaned_list

In [ ]:
# Define lambdas for transformations
lower_index = lambda x: x.lower() if isinstance(x, str) else x
erase_indent = lambda x: x.replace('-', '') if isinstance(x, str) else x
strip_index = lambda x: x.strip() if isinstance(x, str) else x
accents_strip = lambda x: unidecode(x) if isinstance(x, str) else x  # Apply unidecode only to string values
erase_apostrophe = lambda x: x.replace("'", " ") if isinstance(x, str) else x

# Define function to decode string using multiple encodings
def decode_string(original_string):
    if isinstance(original_string, int):  # Check if input is an integer (likely an Excel date)
        return original_string  # Return the integer as is
    elif isinstance(original_string, float):  # Check if input is a float
        return original_string  # Return the float as is
    else:
        encodings_to_try = ['utf-8', 'latin-1', 'windows-1252']
        encoded_string = original_string.encode('utf-8')  # Encode once before the loop
        for encoding in encodings_to_try:
            try:
                decoded_text = encoded_string.decode(encoding)
                return decoded_text
            except UnicodeDecodeError:
                print(f"Failed to decode with {encoding}")
        return original_string  # Return original value if all decoding attempts fail

# Identify columns with dtype 'object' (string columns)
str_columns_list = [str_column for str_column in raw_list.columns if raw_list[str_column].dtype == 'object']

# Apply transformations to selected string columns
for column in str_columns_list:
    cleaned_list[column] = raw_list[column].apply(decode_string).apply(lower_index).apply(erase_indent).apply(strip_index).apply(accents_strip).apply(erase_apostrophe)

# Output the transformed DataFrame
cleaned_list



In [73]:
#manage columns name format

columns_unidecoded = []

for col_name in cleaned_list.columns:
    columns_unidecoded.append(unidecode(col_name).replace('.', ''))

cleaned_list.columns = columns_unidecoded 

cleaned_list


,Lieu accueil,Entrepreneur,Lieu de suivi,Conseiller,Raison sociale,Date de creation,Type de creation,Statut fiscal,Banque,Expert comptable,...,total ante (h),total ante Starter (h),total h suivi,CAPE,Duree (h) consulting,Duree (h) formation crea,partenaire,ref Entreprise,Name ref,Firstname ref
0,unknown,unknown unknown,tourcoing,francois vanoverberghe,a renovation,2022-05-16,creation (activite principale),micro entreprise ir,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,vanoverberghe francois,vanoverberghe,francois
1,unknown,unknown unknown,lens,tiphaine babin,alexandra vanherreweghe,2021-04-19,creation (activite principale),micro entreprise ir,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,babin tiphaine,babin,tiphaine
2,unknown,unknown unknown,denain,olivier ruysschaert,aurelie bouchez consulting,2022-07-01,creation (activite principale),reel is,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,ruysschaert olivier,ruysschaert,olivier
3,unknown,unknown unknown,tourcoing,christian mvula tungila,aurelie versaille deco,2023-01-01,creation (activite principale),micro entreprise ir,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,mvula tungila christian,mvula tungila,christian
4,unknown,unknown unknown,tourcoing,christian mvula tungila,aurelie versaille deco,2023-01-01,creation (activite principale),micro entreprise ir,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,mvula tungila christian,mvula tungila,christian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8537,villeneuve d ascq,julie magnier,villeneuve d ascq,blandine liotard,youlie s delices,2021-02-04,creation (activite principale),micro entreprise ir,unknown,unknown,...,21.5,19.0,1.5,unknown,31.0,unknown,pas de partenaires privileges,liotard blandine,liotard,blandine
8538,villeneuve d ascq,rachel. ducourtioux,villeneuve d ascq,blandine liotard,yujarun,2023-09-12,creation (activite principale),reel is,unknown,unknown,...,5.5,5.5,unknown,16062,unknown,unknown,pas de partenaires privileges,liotard blandine,liotard,blandine
8539,villeneuve d ascq,zackaria zairi,villeneuve d ascq,blandine liotard,zack fast,2021-08-18,creation (activite principale),micro entreprise ir,unknown,unknown,...,2.5,2.5,unknown,unknown,unknown,24.0,pas de partenaires privileges,liotard blandine,liotard,blandine
8540,villeneuve d ascq,carole zens mailly,villeneuve d ascq,mimoun chanfouie,zens mailly carole,2022-04-01,creation (activite principale),micro entreprise ir,unknown,unknown,...,7.5,7.5,6.5,unknown,unknown,150.5,pas de partenaires privileges,chanfouie mimoun,chanfouie,mimoun


In [74]:
#erase useless column
columns_to_erase = cleaned_list.iloc[:, 8:17]
mask = cleaned_list.columns.isin(columns_to_erase.columns)
cleaned_list_shortened = cleaned_list.loc[:, ~mask]
cleaned_list_shortened

,Lieu accueil,Entrepreneur,Lieu de suivi,Conseiller,Raison sociale,Date de creation,Type de creation,Statut fiscal,partenaire,ref Entreprise,Name ref,Firstname ref
0,unknown,unknown unknown,tourcoing,francois vanoverberghe,a renovation,2022-05-16,creation (activite principale),micro entreprise ir,unknown,vanoverberghe francois,vanoverberghe,francois
1,unknown,unknown unknown,lens,tiphaine babin,alexandra vanherreweghe,2021-04-19,creation (activite principale),micro entreprise ir,unknown,babin tiphaine,babin,tiphaine
2,unknown,unknown unknown,denain,olivier ruysschaert,aurelie bouchez consulting,2022-07-01,creation (activite principale),reel is,unknown,ruysschaert olivier,ruysschaert,olivier
3,unknown,unknown unknown,tourcoing,christian mvula tungila,aurelie versaille deco,2023-01-01,creation (activite principale),micro entreprise ir,unknown,mvula tungila christian,mvula tungila,christian
4,unknown,unknown unknown,tourcoing,christian mvula tungila,aurelie versaille deco,2023-01-01,creation (activite principale),micro entreprise ir,unknown,mvula tungila christian,mvula tungila,christian
...,...,...,...,...,...,...,...,...,...,...,...,...
8537,villeneuve d ascq,julie magnier,villeneuve d ascq,blandine liotard,youlie s delices,2021-02-04,creation (activite principale),micro entreprise ir,pas de partenaires privileges,liotard blandine,liotard,blandine
8538,villeneuve d ascq,rachel. ducourtioux,villeneuve d ascq,blandine liotard,yujarun,2023-09-12,creation (activite principale),reel is,pas de partenaires privileges,liotard blandine,liotard,blandine
8539,villeneuve d ascq,zackaria zairi,villeneuve d ascq,blandine liotard,zack fast,2021-08-18,creation (activite principale),micro entreprise ir,pas de partenaires privileges,liotard blandine,liotard,blandine
8540,villeneuve d ascq,carole zens mailly,villeneuve d ascq,mimoun chanfouie,zens mailly carole,2022-04-01,creation (activite principale),micro entreprise ir,pas de partenaires privileges,chanfouie mimoun,chanfouie,mimoun


In [72]:
#csv export 
cleaned_list_shortened.to_csv('cleaned_list_biz_started_from2021_until29022024.csv', index = False)